In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [3]:
%%time
df = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
df.head()

CPU times: user 2.25 s, sys: 190 ms, total: 2.44 s
Wall time: 2.45 s


,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,...,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,...,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,...,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,...,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,...,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0


# Preporcess data

In [4]:
from sklearn.model_selection import StratifiedKFold
import category_encoders as ce
from tqdm import tqdm
import numpy as np
import datetime
import itertools

mapping_ord1 = {'Unknown': 0, 'Novice': 1, 'Contributor': 2, 'Expert': 3, 'Master': 4, 'Grandmaster': 5}
mapping_ord2 = {'Unknown': 0, 'Freezing': 1, 'Cold': 2, 'Warm': 3, 'Hot': 4, 'Boiling Hot': 5, 'Lava Hot': 6}
mapping_ord3 = dict([(v, i) for i, v in enumerate(sorted(set(df['ord_3'].fillna("0"))))])
mapping_ord4 = dict([(v, i) for i, v in enumerate(sorted(set(df['ord_4'].fillna("0"))))])
mapping_ord5 = dict([(v, i) for i, v in enumerate(sorted(set(df['ord_5'].fillna("0"))))])


def preprocess_data(df):
    
    df["nom_0_nom_1"] = df["nom_0"] + df["nom_1"]
    df["nom_3_ord_1"] = df["nom_0"] + df["ord_1"]
    
    df['ord_0'] = df['ord_0'].fillna(0)
    df['ord_1'] =  df['ord_1'].fillna('Unknown').map(mapping_ord1)
    df['ord_2'] =  df['ord_2'].fillna('Unknown').map(mapping_ord2)
    df['ord_3'] =  df['ord_3'].fillna('0').map(mapping_ord3)
    df['ord_4'] =  df['ord_4'].fillna('0').map(mapping_ord4)
    df['ord_5'] =  df['ord_5'].fillna('0').map(mapping_ord5)
    df['bin_3'] = df['bin_3'].fillna('U').map({"T": 1, "F": 0, "U": np.nan})
    df['bin_4'] = df['bin_4'].fillna('U').map({"Y": 1, "N": 0, "U": np.nan})

    df['day_sin'] = np.sin(df.day*(2.*np.pi/24))
    df['day_cos'] = np.cos(df.day*(2.*np.pi/24))
    df['month_sin'] = np.sin((df.month-1)*(2.*np.pi/12))
    df['month_cos'] = np.cos((df.month-1)*(2.*np.pi/12))
    
    return df

def encode(df, df_test, cols):
    cols_enc = list(map(lambda x: x + "_enc", cols))
    for c in cols_enc:
        df[c] = np.nan
        df_test[c] = np.nan
        
    skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
    
    for train_idx, oof_idx in tqdm(skf.split(df, df["target"])):
        enc = ce.TargetEncoder(cols = cols, smoothing=0.3)
        enc.fit(df.loc[train_idx, cols], df.loc[train_idx, "target"])
        df.loc[oof_idx, cols_enc] = enc.transform(df.loc[oof_idx, cols]).values
    
    enc = ce.TargetEncoder(cols = cols, smoothing=0.3)
    enc.fit(df[cols], df["target"])
    df_test[cols_enc] = enc.transform(df_test[cols])
    return df, df_test, cols_enc
    
df = preprocess_data(df)
df_test = preprocess_data(df_test)

binary = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']
ordinal  = ['ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']
low_card = ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4']
high_card = ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', "nom_0_nom_1", "nom_3_ord_1"]
date = ['day', 'month']
cyclic = ['day_sin', 'day_cos', 'month_sin', 'month_cos']

df, df_test, cat_enc =  encode(df, df_test, high_card + low_card + date)
features = binary + ordinal + cat_enc + cyclic

10it [00:39,  3.93s/it]


In [5]:
df.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9_enc,nom_0_nom_1_enc,nom_3_ord_1_enc,nom_0_enc,nom_1_enc,nom_2_enc,nom_3_enc,nom_4_enc,day_enc,month_enc
0,0,0.0,0.0,0.0,0.0,0.0,Red,Trapezoid,Hamster,Russia,...,0.171123,0.219889,0.168903,0.183116,0.225354,0.168057,0.219869,0.207528,0.200237,0.146438
1,1,1.0,1.0,0.0,0.0,1.0,Red,Star,Axolotl,NaN,...,0.147679,0.122500,0.228065,0.183024,0.130305,0.200561,0.185587,0.179620,0.212574,0.208655
2,2,0.0,1.0,0.0,0.0,0.0,Red,NaN,Hamster,Canada,...,0.188596,0.183130,0.184976,0.183116,0.181924,0.168057,0.160406,0.207528,0.163968,0.212740
3,3,NaN,0.0,0.0,0.0,0.0,Red,Circle,Hamster,Finland,...,0.269231,0.176106,0.151445,0.183027,0.179732,0.168284,0.179400,0.179624,0.163910,0.146364
4,4,0.0,NaN,0.0,1.0,0.0,Red,Triangle,Hamster,Costa Rica,...,0.255446,0.154720,0.228840,0.183206,0.157499,0.167818,0.202551,0.187077,0.163213,0.225034


In [6]:
df.shape

(600000, 45)

# Learn model

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)

        try:
            return X[self.columns]
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("The DataFrame does not include the columns: %s" % cols_error)

In [8]:
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
import category_encoders as ce
 
params = {
        'clf__objective': 'binary',
        'clf__boosting_type': 'gbdt', 
        'clf__metric': 'auc',
        'clf__learning_rate': 0.01,
        "clf__bagging_freq": 5,
        "clf__bagging_fraction": 0.8,
        "clf__min_data_in_leaf": 30,
        "clf__min_sum_hessian_in_leaf": 5,
        "clf__use_two_round_loading": False,
        "clf__feature_fraction": 0.8,
        'clf__verbose': 1,
        "clf__lambda_l1": 0.1,
        "clf__n_estimators": 5000,
        "clf__is_unbalance": True
    }

                        
pipeline = Pipeline(steps=[('sel', ColumnSelector(features)), 
                           ('clf', LGBMClassifier())]).set_params(**params)

In [9]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt.pyll.base import scope
from hyperopt import space_eval
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


def make_objective(pipeline, df):
    
    def objective(params):
                
        pipeline.set_params(**params)
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        scores = []
        for train_idx, val_idx in tqdm(skf.split(df, df["target"])):
            
            df_train, df_val = df.loc[train_idx, :], df.loc[val_idx, :]
            transformers = Pipeline(pipeline.steps[:-1]).fit(df_train, df_train['target'])
            X_val, y_val = transformers.transform(df_val), df_val['target']
            
            fit_args = {
                "clf__early_stopping_rounds": 500, 
                "clf__eval_set": (X_val, y_val),
                "clf__verbose": 500,
                "clf__eval_metric": 'auc',
            }
            
            pipeline.fit(df_train, df_train['target'], **fit_args)
            prob = pipeline.predict_proba(df_val)[:, 1]
            scores.append(roc_auc_score(y_val, prob))
            
        score = np.mean(scores)
        print("AUC {:.5f}+-{:.5f}.\n".format(np.mean(scores), np.std(scores)) + 
              "Params: \n" + 
              "\n".join(["{}: {:.3f}".format(k, params[k]) for k in params]))
        return 1 - score
    
    return objective

space = {
        "clf__max_depth": hp.choice("clf__max_depth", [1,2,3,4,5,6]),
        "clf__learning_rate": hp.loguniform("clf__learning_rate", np.log(0.005), np.log(0.5)),
        'clf__lambda_l1': hp.loguniform('clf__lambda_l1', np.log(0.005), np.log(0.5)),
        "clf__num_leaves": hp.choice("clf__num_leaves", np.linspace(8, 128, 50, dtype=int)),
        "clf__feature_fraction": hp.quniform("clf__feature_fraction", 0.5, 1.0, 0.1),
        "clf__bagging_fraction": hp.quniform("clf__bagging_fraction", 0.5, 1.0, 0.1),
        "clf__min_child_weight": hp.uniform('clf__min_child_weight', 0.5, 10),
        'clf__colsample_bytree': hp.uniform('clf__colsample_bytree', 0.3, 1.0),
    }

best = fmin(fn=make_objective(pipeline, df),
            space=space,
            algo=tpe.suggest,
            max_evals=20)

best_params = space_eval(space, best)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

0it [00:00, ?it/s]



Training until validation scores don't improve for 500 rounds
[500]	valid_0's auc: 0.754427                         
[1000]	valid_0's auc: 0.771993                        
[1500]	valid_0's auc: 0.77842                         
[2000]	valid_0's auc: 0.781569                        
[2500]	valid_0's auc: 0.783919                        
[3000]	valid_0's auc: 0.785282                        
[3500]	valid_0's auc: 0.786194                        
[4000]	valid_0's auc: 0.786803                        
[4500]	valid_0's auc: 0.7872                          
[5000]	valid_0's auc: 0.787445                        
Did not meet early stopping. Best iteration is:       
[4976]	valid_0's auc: 0.787446
  0%|          | 0/20 [00:43<?, ?trial/s, best loss=?]

1it [00:44, 44.66s/it]



Training until validation scores don't improve for 500 rounds
[500]	valid_0's auc: 0.755407                         
[1000]	valid_0's auc: 0.773913                        
[1500]	valid_0's auc: 0.780216                        
[2000]	valid_0's auc: 0.783447                        
[2500]	valid_0's auc: 0.785753                        
[3000]	valid_0's auc: 0.787231                        
[3500]	valid_0's auc: 0.788183                        
[4000]	valid_0's auc: 0.788849                        
[4500]	valid_0's auc: 0.789246                        
[5000]	valid_0's auc: 0.789532                        
Did not meet early stopping. Best iteration is:       
[4990]	valid_0's auc: 0.789536
  0%|          | 0/20 [01:32<?, ?trial/s, best loss=?]

...

Training until validation scores don't improve for 500 rounds                      
[500]	valid_0's auc: 0.785658                                                      
[1000]	valid_0's auc: 0.785372                                                     
Early stopping, best iteration is:                                                   
[563]	valid_0's auc: 0.785736
 95%|█████████▌| 19/20 [1:00:08<04:02, 242.78s/trial, best loss: 0.21125288283071575]

4it [01:24, 21.57s/it]



Training until validation scores don't improve for 500 rounds                        
[500]	valid_0's auc: 0.785396                                                        
[1000]	valid_0's auc: 0.785558                                                       
Early stopping, best iteration is:                                                   
[710]	valid_0's auc: 0.785638
 95%|█████████▌| 19/20 [1:00:29<04:02, 242.78s/trial, best loss: 0.21125288283071575]

5it [01:45, 21.54s/it]

5it [01:45, 21.19s/it]


AUC 0.78695+-0.00122.                                                                
Params: 
clf__bagging_fraction: 0.800
clf__colsample_bytree: 0.908
clf__feature_fraction: 1.000
clf__lambda_l1: 0.146
clf__learning_rate: 0.047
clf__max_depth: 4.000
clf__min_child_weight: 0.628
clf__num_leaves: 32.000
100%|██████████| 20/20 [1:00:30<00:00, 181.53s/trial, best loss: 0.21125288283071575]


In [10]:
print("Best params: \n" + "\n".join(["{}: {:.3f}".format(k, best_params[k]) for k in best_params]))

Best params: 
clf__bagging_fraction: 0.600
clf__colsample_bytree: 0.307
clf__feature_fraction: 0.800
clf__lambda_l1: 0.006
clf__learning_rate: 0.063
clf__max_depth: 1.000
clf__min_child_weight: 1.473
clf__num_leaves: 110.000


In [11]:
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

folds = 50
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
pipeline.set_params(**best_params)
df_test['target'] = 0

for train_idx, val_idx in tqdm(skf.split(df, df['target'])):
    df_train = df.loc[train_idx, :]
    df_val = df.loc[val_idx, :]
    X_val, y_val = Pipeline(pipeline.steps[:-1]).fit(df_train, df_train['target'])\
                                                    .transform(df_val), df_val['target']
    fit_args = {
        "clf__early_stopping_rounds": 500, 
        "clf__eval_set": (X_val, y_val),
        "clf__verbose": 500,
        "clf__eval_metric": 'auc'
    }
        
    pipeline.fit(df_train, df_train['target'], **fit_args)
    prob = pipeline.predict_proba(df_val)[:, 1]
    score = roc_auc_score(y_val, prob)
    
    df_test['target'] += pipeline.predict_proba(df_test)[:, 1]


0it [00:00, ?it/s]

Training until validation scores don't improve for 500 rounds
[500]	valid_0's auc: 0.783978
[1000]	valid_0's auc: 0.789184
[1500]	valid_0's auc: 0.790144
[2000]	valid_0's auc: 0.790417
[2500]	valid_0's auc: 0.790582
[3000]	valid_0's auc: 0.79059
Early stopping, best iteration is:
[2757]	valid_0's auc: 0.79065


1it [00:23, 23.09s/it]

Training until validation scores don't improve for 500 rounds
[500]	valid_0's auc: 0.786184
[1000]	valid_0's auc: 0.792877
[1500]	valid_0's auc: 0.794003
[2000]	valid_0's auc: 0.794163
[2500]	valid_0's auc: 0.794166
Early stopping, best iteration is:
[2358]	valid_0's auc: 0.794244


2it [00:44, 22.52s/it]

...

48it [21:31, 22.93s/it]

Training until validation scores don't improve for 500 rounds
[500]	valid_0's auc: 0.778251
[1000]	valid_0's auc: 0.785687
[1500]	valid_0's auc: 0.787702
[2000]	valid_0's auc: 0.78822
[2500]	valid_0's auc: 0.788405
[3000]	valid_0's auc: 0.788563
Early stopping, best iteration is:
[2917]	valid_0's auc: 0.788657


49it [21:54, 23.05s/it]

Training until validation scores don't improve for 500 rounds
[500]	valid_0's auc: 0.776445
[1000]	valid_0's auc: 0.782818
[1500]	valid_0's auc: 0.785007
[2000]	valid_0's auc: 0.785363
[2500]	valid_0's auc: 0.785692
[3000]	valid_0's auc: 0.785595
Early stopping, best iteration is:
[2558]	valid_0's auc: 0.785707


50it [22:15, 26.71s/it]


In [12]:
df_test['target'] /= folds
df_test[['id', 'target']].to_csv('../data/submission_lgbm.csv', index=False)
!head -n 5 ../data/submission_lgbm.csv

id,target
600000,0.374122485210922
600001,0.5983712415087725
600002,0.4304930025513145
600003,0.3404663496448673
